In [6]:
!git clone https://github.com/a3kc74/TOXIC-COMMENT-DETECTION.git

Cloning into 'TOXIC-COMMENT-DETECTION'...
fatal: unable to access 'https://github.com/a3kc74/TOXIC-COMMENT-DETECTION.git/': Could not resolve host: github.com


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as wr
import pyarrow.parquet as pq
import tools.utils as utils
from sklearn.feature_extraction.text import CountVectorizer
wr.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tools'

In [ ]:
# df = pd.read_parquet('data/train-00000-of-00002.parquet')
table = pq.read_table('data/cleaned_jigsaw.parquet')
df = table.to_pandas()
print(df.head())

In [ ]:
# Cell 2: Label distribution

label_cols = ["non_toxic", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Số lượng "1" (positive) cho từng nhãn
label_counts = df[label_cols].ne(0).sum()

# Tỷ lệ positive (so với tổng số dòng)
label_rates = label_counts / len(df)

print("Số lượng positive mỗi nhãn:")
print(label_counts)
print("\nTỷ lệ positive mỗi nhãn:")
print(label_rates)

# Vẽ bar chart
plt.figure(figsize=(8, 5))
plt.bar(label_cols, label_rates)
plt.title("Label Positive Rate for cleaned_jigsaw.parquet (Imbalance Check)")
plt.ylabel("Positive rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [25]:
# Cell 3: Length statistics

# Độ dài comment theo số từ
df["comment_length"] = df["comment_text"].str.split().str.len()

# Nhóm theo non_toxic
length_stats = df.groupby("non_toxic")["comment_length"].agg(["mean", "median"])
length_stats.index = ["toxic (0)", "non_toxic (1)"]
print(length_stats)

# Vẽ biểu đồ so sánh độ dài trung bình
plt.figure(figsize=(6, 4))
length_stats["mean"].plot(kind="bar")
plt.title("Average Comment Length: Toxic vs Non-toxic for cleaned_jigsaw.parquet")
plt.ylabel("Average number of tokens")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# Cell 5: Split groups

# Toxic group = non_toxic == 0
toxic_mask = df["non_toxic"] == 0
non_toxic_mask = df["non_toxic"] == 1

toxic_texts = df.loc[toxic_mask, "comment_text"]
non_toxic_texts = df.loc[non_toxic_mask, "comment_text"]

print("Toxic comments:", len(toxic_texts))
print("Non-toxic comments:", len(non_toxic_texts))

In [ ]:
# Cell 6: Top N unigrams (words)

TOP_K = 20

top_words_toxic = utils.get_top_ngrams(toxic_texts, ngram_range=(1,1), top_k=TOP_K)
top_words_non_toxic = utils.get_top_ngrams(non_toxic_texts, ngram_range=(1,1), top_k=TOP_K)

print("Top words - TOXIC:")
for w, c in top_words_toxic:
    print(f"{w}: {c}")

print("\nTop words - NON-TOXIC:")
for w, c in top_words_non_toxic:
    print(f"{w}: {c}")

In [ ]:
# Bar chart top words toxic

words, counts = zip(*top_words_toxic)

plt.figure(figsize=(10, 5))
plt.bar(words, counts)
plt.title("Top Unigrams in TOXIC Comments")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


In [ ]:
words, counts = zip(*top_words_non_toxic)

plt.figure(figsize=(10, 5))
plt.bar(words, counts)
plt.title("Top Unigrams in NON-TOXIC Comments")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Cell 7: Top N bigrams

TOP_K = 20

top_bigrams_toxic = utils.get_top_ngrams(toxic_texts, ngram_range=(2,2), top_k=TOP_K)
top_bigrams_non_toxic = utils.get_top_ngrams(non_toxic_texts, ngram_range=(2,2), top_k=TOP_K)

print("Top bigrams - TOXIC:")
for w, c in top_bigrams_toxic:
    print(f"{w}: {c}")

print("\nTop bigrams - NON-TOXIC:")
for w, c in top_bigrams_non_toxic:
    print(f"{w}: {c}")

In [ ]:
# Bar chart top bigrams toxic
bigrams, counts = zip(*top_bigrams_toxic)

plt.figure(figsize=(10, 5))
plt.bar(bigrams, counts)
plt.title("Top Bigrams in TOXIC Comments")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [22]:
target_labels = ["severe_toxic", "obscene", "threat", "insult", "identity_hate"]
TOP_K = 20

for label in target_labels:
    # Chọn các comment có nhãn này (label > 0)
    texts = df.loc[df[label] > 0, "comment_text"]
    
    # Optional: sample nếu quá lớn để nhanh hơn
    max_samples = 100000
    if len(texts) > max_samples:
        texts = texts.sample(max_samples, random_state=42)

    top_unigrams = utils.get_top_ngrams(texts, ngram_range=(1, 1), top_k=TOP_K)
    print(f"\n===== Top {TOP_K} UNIGRAMS for label = {label} =====")
    for w, c in top_unigrams:
        print(f"{w}: {c}")
    
    # Vẽ bar chart
    words, counts = zip(*top_unigrams)
    
    plt.figure(figsize=(10, 5))
    plt.bar(words, counts)
    plt.title(f"Top {TOP_K} Unigrams for label = {label}")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()


In [24]:
target_labels = ["severe_toxic", "obscene", "threat", "insult", "identity_hate"]
TOP_K = 20

for label in target_labels:
    texts = df.loc[df[label] > 0, "comment_text"]
    
    max_samples = 100000
    if len(texts) > max_samples:
        texts = texts.sample(max_samples, random_state=42)
    
    top_bigrams = utils.get_top_ngrams(texts, ngram_range=(2, 2), top_k=TOP_K)
    print(f"\n===== Top {TOP_K} BIGRAMS for label = {label} =====")
    for w, c in top_bigrams:
        print(f"{w}: {c}")
    
    bigrams, counts = zip(*top_bigrams)
    
    plt.figure(figsize=(10, 5))
    plt.bar(bigrams, counts)
    plt.title(f"Top {TOP_K} Bigrams for label = {label}")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()